In [ ]:
import sys
print(sys.executable)
print(sys.path)

In [ ]:
import findspark
findspark.init() #This line initializes findspark.

In [23]:
import os
os.environ['SPARK_HOME'] = "/opt/homebrew/Cellar/apache-spark/3.5.4/libexec" #Or your spark home.
import findspark
findspark.init()

In [24]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as F
#from pyspark.sql import types as T
from pyspark.sql.functions import col
from pyspark.sql import types
#from deep_translator import GoogleTranslator

In [25]:
credentials_location = './gcs.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location) \
    .set("spark.driver.extraClassPath", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.executor.extraClassPath", "./lib/gcs-connector-hadoop3-2.2.5.jar")

In [28]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [27]:
spark.stop()

In [29]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

#.config(conf=sc.getConf()) \
#.config("spark.sql.legacy.timeParserPolicy", "LEGACY") \

# Courses

In [31]:
df_courses = spark.read.option("header", "true").parquet('gs://jugnu-france-course-enrollments/courses_enrol_data_2025_03_24/courses_raw_parquet/*.parquet')

In [32]:
df_courses.show(5)

25/03/24 13:39:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+------------+------------------+--------------------+----------------+---------------+---------+----------------------+-------------------------------+----------------+----------------+----------------+----------------+----------------+--------------------------------+-----------+-----------+-----------+-----------+-----------+--------------------+-------------+-------------+----------+----------+----------+---------------+--------------+--------------------+------------------+--------------------+--------------------+--------------------+----------------------------+---------+-----------------+---------------------+-----------------------+-----------------------+------------------------+-----------------+-----------------+------------------+----------------+-----------+-----------------+-----------------+
|date_extract|      nom_of|   nom_departement|          nom_region|type_referentiel|code_inventaire|code_rncp|intitule_certification|libelle_niveau_sortie_formation|cod

In [ ]:
df_courses.schema

In [ ]:
# schema = types.StructType([
#     types.StructField('date_extract', types.StringType(), True), 
#     types.StructField('nom_of', types.StringType(), True), 
#     types.StructField('nom_departement', types.StringType(), True), 
#     types.StructField('nom_region', types.StringType(), True), 
#     types.StructField('type_referentiel', types.StringType(), True), 
#     types.StructField('code_inventaire', types.LongType(), True), 
#     types.StructField('code_rncp', types.LongType(), True), 
#     types.StructField('intitule_certification', types.StringType(), True), 
#     types.StructField('libelle_niveau_sortie_formation', types.StringType(), True), 
#     types.StructField('code_formacode_1', types.DoubleType(), True), 
#     types.StructField('code_formacode_2', types.DoubleType(), True), 
#     types.StructField('code_formacode_3', types.DoubleType(), True), 
#     types.StructField('code_formacode_4', types.DoubleType(), True), 
#     types.StructField('code_formacode_5', types.DoubleType(), True), 
#     types.StructField('libelle_code_formacode_principal', types.StringType(), True), 
#     types.StructField('code_rome_1', types.StringType(), True), 
#     types.StructField('code_rome_2', types.StringType(), True), 
#     types.StructField('code_rome_3', types.StringType(), True), 
#     types.StructField('code_rome_4', types.StringType(), True), 
#     types.StructField('code_rome_5', types.StringType(), True), 
#     types.StructField('libelle_nsf_1', types.StringType(), True), 
#     types.StructField('libelle_nsf_2', types.StringType(), True), 
#     types.StructField('libelle_nsf_3', types.StringType(), True), 
#     types.StructField('code_nsf_1', types.StringType(), True), 
#     types.StructField('code_nsf_2', types.StringType(), True), 
#     types.StructField('code_nsf_3', types.StringType(), True), 
#     types.StructField('code_certifinfo', types.StringType(), True), 
#     types.StructField('siret', types.StringType(), True), 
#     types.StructField('numero_formation', types.StringType(), True), 
#     types.StructField('intitule_formation', types.StringType(), True), 
#     types.StructField('points_forts', types.StringType(), True), 
#     types.StructField('objectif_formation', types.StringType(), True), 
#     types.StructField('contenu_formation', types.StringType(), True), 
#     types.StructField('resultats_attendus_formation', types.StringType(), True), 
#     types.StructField('nb_action', types.IntegerType(), True), 
#     types.StructField('nb_session_active', types.IntegerType(), True), 
#     types.StructField('nb_session_a_distance', types.IntegerType(), True), 
#     types.StructField('nombre_heures_total_min', types.IntegerType(), True), 
#     types.StructField('nombre_heures_total_max', types.IntegerType(), True), 
#     types.StructField('nombre_heures_total_mean', types.IntegerType(), True), 
#     types.StructField('frais_ttc_tot_min', types.FloatType(), True), 
#     types.StructField('frais_ttc_tot_max', types.FloatType(), True), 
#     types.StructField('frais_ttc_tot_mean', types.FloatType(), True), 
#     types.StructField('code_departement', types.StringType(), True), 
#     types.StructField('code_region', types.StringType(), True), 
#     types.StructField('nbaction_nbheures', types.IntegerType(), True), 
#     types.StructField('coderegion_export', types.StringType(), True)
#     ])

In [ ]:
# df_courses_s = spark.read.option("header", "true").schema(schema).parquet('gs://jugnu-france-course-enrollments/courses_data/courses_raw_parquet/*.parquet')

In [35]:
# Cast the 'code_inventaire' column to a string
df_courses = df_courses.withColumn("code_inventaire", df_courses["code_inventaire"].cast(types.StringType()))\
    .withColumn("code_rncp", df_courses["code_rncp"].cast(types.StringType()))\
    .withColumn("code_formacode_1", F.regexp_replace(df_courses["code_formacode_1"].cast(types.StringType()), r'\.0$', '')) \
    .withColumn("code_formacode_2", F.regexp_replace(df_courses["code_formacode_2"].cast(types.StringType()), r'\.0$', '')) \
    .withColumn("code_formacode_3", F.regexp_replace(df_courses["code_formacode_3"].cast(types.StringType()), r'\.0$', '')) \
    .withColumn("code_formacode_4", F.regexp_replace(df_courses["code_formacode_4"].cast(types.StringType()), r'\.0$', '')) \
    .withColumn("code_formacode_5", F.regexp_replace(df_courses["code_formacode_5"].cast(types.StringType()), r'\.0$', '')) \
    .withColumn("code_nsf_1", df_courses["code_nsf_1"].cast(types.StringType()))\
    .withColumn("code_nsf_2", df_courses["code_nsf_2"].cast(types.StringType()))\
    .withColumn("code_nsf_3", df_courses["code_nsf_3"].cast(types.StringType()))\
    .withColumn("code_certifinfo", df_courses["code_certifinfo"].cast(types.StringType()))\
    .withColumn("siret", df_courses["siret"].cast(types.StringType()))\
    .withColumn("cost_max", df_courses["frais_ttc_tot_max"].cast(types.DecimalType(10, 2)))\
    .withColumn("cost_min", df_courses["frais_ttc_tot_min"].cast(types.DecimalType(10, 2)))\
    .withColumn("cost_mean", df_courses["frais_ttc_tot_mean"].cast(types.DecimalType(10, 2)))

In [36]:
df_courses.head(2)

[Row(date_extract='2025-03-24', nom_of='BT FORMATION', nom_departement='Haute-Loire', nom_region='Auvergne-Rhône-Alpes', type_referentiel='RS', code_inventaire='6965', code_rncp='-1', intitule_certification='Créer et gérer des sites web avec WordPress (Tosa)', libelle_niveau_sortie_formation=None, code_formacode_1=None, code_formacode_2=None, code_formacode_3=None, code_formacode_4=None, code_formacode_5=None, libelle_code_formacode_principal=None, code_rome_1=None, code_rome_2=None, code_rome_3=None, code_rome_4=None, code_rome_5=None, libelle_nsf_1="Informatique, traitement de l'information, réseaux de transmission", libelle_nsf_2=None, libelle_nsf_3=None, code_nsf_1='326.0', code_nsf_2=None, code_nsf_3=None, code_certifinfo='-1', siret='48121600000044', numero_formation='48121600000044_11172-60', intitule_formation='LOGICIEL WORDPRESS', points_forts='<p>24 ans d\'expérience à votre service.</p><p>- Durée, rythme selon vos disponibilités.</p><p>- Préparation et passage à la certifica

In [ ]:
# Convert the string to a timestamp with the correct format
df_courses_data = df_courses.withColumn('date_extract', F.to_date(F.col('date_extract'), 'yyyy-MM-dd'))


In [ ]:
df_courses_data.head(2)

In [ ]:
# Assuming your DataFrame is named df

filter_values = [
    "Informatique, traitement de l'information, réseaux de transmission",
    "Enseignement, formation",
    "Commerce, vente",
    "Comptabilite, gestion",
    "Spécialités pluri-scientifiques",
    "Spécialites plurivalentes de la communication et de l'information"
]

In [ ]:
df_courses_data = df_courses_data.filter(col("libelle_nsf_1").isin(filter_values))

In [ ]:
df_courses.count()

In [ ]:
# Define the columns to rename and their new names
columns_to_rename = {
    'nom_of': 'provider',
    'siret': 'provider_ID',
    'nom_region': 'region',
    'nom_departement': 'department',
    'intitule_certification': 'certification_title',
    'libelle_niveau_sortie_formation': 'training_exit_level',
    'libelle_code_formacode_principal': 'main_formacode_desc',
    'libelle_nsf_1': 'nsf_code_1_desc',
    'libelle_nsf_2': 'nsf_code_2_desc',
    'libelle_nsf_3': 'nsf_code_3_desc',
    'numero_formation': 'training_ID',
    'intitule_formation': 'title',
    'points_forts': 'strengths',
    'nb_session_active': 'nb_active_session',
    'nb_session_a_distance': 'nb_distant_session',
    'nombre_heures_total_min': 'duration_min',
    'nombre_heures_total_max': 'duration_max',
    'nombre_heures_total_mean': 'duration_mean',
    'frais_ttc_tot_min': 'cost_min',
    'frais_ttc_tot_max': 'cost_max',
    'frais_ttc_tot_mean': 'cost_mean'
}

In [ ]:
# Rename the columns
for old_name, new_name in columns_to_rename.items():
    if old_name in df_courses_data.columns:
        df_courses_data = df_courses_data.withColumnRenamed(old_name, new_name)
    else:
        print(f"Column '{old_name}' not found, skipping rename.")

In [ ]:
df_courses.columns

In [ ]:
df_courses_data.schema

In [ ]:
df_courses_data.coalesce(1).write.parquet('gs://jugnu-france-course-enrollments/courses_enrol_data_2025_03_20/courses_filtered', mode='overwrite')

S.unique_row_id, S.date_extract, S.provider, S.department, S.region, S.type_referential, S.code_rs, S.code_rncp, S.certification_title, S.training_exit_level, S.code_formacode_1, S.code_formacode_2, S.code_formacode_3, S.code_formacode_4, S.code_formacode_5, S.main_formacode_desc, S.code_rome_1, S.code_rome_2, S.code_rome_3, S.code_rome_4, S.code_rome_5, S.nsf_code_1_desc, S.nsf_code_2_desc, S.nsf_code_3_desc, S.nsf_code_1, S.nsf_code_2, S.nsf_code_3, S.code_certification, S.provider_id, S.training_id, S.training_title, S.strong_points, S.training_objective, S.training_content, S.expected_training_results, S.trianing_module_count, S.nb_session_active, S.nb_remote_session, S.duration_min, S.duration_max, S.duration_mean, S.fees_min, S.fees_max, S.fees_mean, S.department_code, S.region_code, S.nb_hours, S.coderegion_export

# earlier

In [ ]:
def translate_batch(unique_values):
    translations = {}
    translator = GoogleTranslator(source='auto', target='en')
    for value in unique_values:
        if value is not None:
            try:
                translations[value] = translator.translate(value)
            except Exception as e:
                print(f"Translation error: {e}")
                translations[value] = None
        else:
            translations[value] = None
    return translations

In [ ]:
columns_to_translate = ['certification_title'] #, 'title', 'main_formacode_desc']
translation_maps = {}

In [ ]:
print(f"Number of partitions: {df_courses.rdd.getNumPartitions()}")

In [ ]:
df_courses = df_courses.repartition(24)

In [ ]:
for col_name in columns_to_translate:
    unique_values = [row[0] for row in df_courses.select(col_name).distinct().collect()]
    translation_maps[col_name] = translate_batch(unique_values)

In [ ]:
for col_name in columns_to_translate:
    translation_map = translation_maps[col_name]
    broadcasted_map = sc.broadcast(translation_map)

    @F.udf(returnType=T.StringType())
    def lookup_translation(input_value):
        return broadcasted_map.value.get(input_value)

    df_courses = df_courses.withColumn(f'{col_name}_en', lookup_translation(F.col(col_name)))

In [ ]:
@F.udf(returnType=T.StringType())
def translate(input):
    if input is None:
        return None  # Or return "" if you prefer an empty string
    try:
        return GoogleTranslator(source='auto', target='en').translate(input)
    except NotValidPayload:
        return None #Or some other error handling.
    except Exception as e:
        print(f"Translation error: {e}")
        return None

In [ ]:
df_courses = df_courses\
            .withColumn('certification_title_en', translate(F.col('certification_title')))\
            .withColumn('title_en', translate(F.col('title')))\
            .withColumn('main_formacode_desc_en', translate(F.col('main_formacode_desc')))

In [ ]:
df_courses.show(5)

In [ ]:
df_courses\
    .write\
    .parquet('gs://jugnu-france-course-enrollments/courses_data/courses_raw_parquet/france_courses_en.parquet', mode='overwrite')

In [ ]:
import pandas as pd
import gcsfs
import os

def read_parquet_from_gcs(gcs_path, credentials_path):
    """Reads a Parquet file from Google Cloud Storage into a pandas DataFrame."""
    try:
        # Set the environment variable within the function
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gcs.json"

        fs = gcsfs.GCSFileSystem()
        with fs.open(gcs_path) as f:
            df = pd.read_parquet(f, )
        return df
    except Exception as e:
        print(f"Error reading Parquet file: {e}")
        return None

# Example usage:
gcs_path = "gs://your-bucket/path/to/your/file.parquet"  # Replace with your GCS path
credentials_path = "gcs.json"  # Replace with your path

df = read_parquet_from_gcs(gcs_path, credentials_path)

if df is not None:
    print(df.head())

# Enrollments

In [ ]:
df_enrollments = spark.read.option("header", "true").parquet('gs://jugnu-france-course-enrollments/courses_enrol_data_2025_03_21/enrollments_raw_parquet/*.parquet')

In [ ]:
df_enrollments.show()

In [ ]:
df_enrollments.count()

In [ ]:
df_enrollments.schema

In [ ]:
# Cast the 'code_inventaire' column to a string
df_enrollments = df_enrollments\
    .withColumn('annee_mois', F.to_date(df_enrollments["annee_mois"], "yyyy-MM"))\
    .withColumn("annee", df_enrollments["annee"].cast(types.StringType()))\
    .withColumn("mois", df_enrollments["mois"].cast(types.StringType()))\
    .withColumn("code_rncp", df_enrollments["code_rncp"].cast(types.StringType()))\
    .withColumn("code_rs", df_enrollments["code_rs"].cast(types.StringType()))\
    .withColumn("code_certifinfo", df_enrollments["code_certifinfo"].cast(types.StringType()))\
    .withColumn("siret_of_contractant", df_enrollments["siret_of_contractant"].cast(types.StringType()))\
    .withColumn("date_chargement", F.to_date(df_enrollments["date_chargement"], "yyyy-MM-dd"))
    

In [ ]:
df_enrollments.schema

In [ ]:
df_enrollments.show()

In [ ]:
# Define the columns to rename and their new names
columns_to_rename = {
        'annee_mois': 'year_month',
        'annee': 'year',
        'mois': 'month',
        'type_referentiel': 'type_referential',
        'code_certifinfo': 'code_certification',
        'intitule_certification': 'certification_title',
        'siret_of_contractant': 'provider_ID',
        'raison_sociale_of_contractant': 'provider',
        'entrees_formation': 'training_entries',
        'sorties_realisation_partielle': 'partial_completion_exits',
        'sorties_realisation_totale': 'total_completion_exits',
        'date_chargement': 'load_date'
    }

In [ ]:
# Rename the columns
for old_name, new_name in columns_to_rename.items():
    if old_name in df_enrollments.columns:
        df_enrollments = df_enrollments.withColumnRenamed(old_name, new_name)
    else:
        print(f"Column '{old_name}' not found, skipping rename.")

In [ ]:
df_enrollments.coalesce(1).write.parquet('gs://jugnu-france-course-enrollments/courses_enrol_data_2025_03_21/enrollments_filtered', mode='overwrite')

In [ ]:
df_enrollments["year_month"]

In [ ]:
S.year_month, S.year, S.month, S.type_referential, S.code_rncp, S.code_rs, S.code_certification, S.certification_title, S.provider_id,
S.provider, S.partial_achievement_exits, S.complete_achievement_exits, S.load_date

# Formacode Translation

In [ ]:
import os
os.environ['SPARK_HOME'] = "/opt/homebrew/Cellar/apache-spark/3.5.4/libexec" #Or your spark home.
import findspark
findspark.init()

In [1]:
#import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as F
from pyspark.sql import types as T
#from pyspark.sql.functions import col
#from pyspark.sql import types
from deep_translator import GoogleTranslator
import time

In [2]:
credentials_location = './gcs.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test-2') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location) \
    .set("spark.driver.extraClassPath", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.executor.extraClassPath", "./lib/gcs-connector-hadoop3-2.2.5.jar")

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

25/03/24 10:52:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

#.config(conf=sc.getConf()) \
#.config("spark.sql.legacy.timeParserPolicy", "LEGACY") \

In [ ]:
#from pyspark.sql import functions as F
#from pyspark.sql import types as T

In [ ]:
# from pyspark.sql import functions as F
# from pyspark.sql import types as T

# spark = ...

# df = spark.createDataFrame([
#     ['ich suche den namen'],
#     ['guten tag']],
#     ['name'])

# @F.udf(returnType=T.StringType())
# def translate(input):
#    # from deep_translator import GoogleTranslator
#     return GoogleTranslator(source='auto', target='en').translate(input)

# df.withColumn('translation', translate(F.col('name'))).show()

In [5]:
df_formacode = spark.read\
                         .option("header", "true") \
                         .csv("./data/formacode_description.csv")

In [6]:
df_formacode.columns

['formacode', 'description', 'field', 'generic_term']

In [7]:
# 1. Replace empty fields
df_formacode = df_formacode.withColumn("field", F.coalesce(F.col("field"), F.lit("000 unknown")))

In [8]:
df_formacode = df_formacode.repartition(16, "field")
#df_formacode = df_formacode.repartition("field")



In [9]:
print(f"Number of partitions: {df_formacode.rdd.getNumPartitions()}")

Number of partitions: 16


In [10]:
def translate(input_str):
    if input_str and len(input_str) > 0: # Check for empty or None
        try:
            #time.sleep(0.3)
            return GoogleTranslator(source='auto', target='en').translate(input_str)
        except Exception as e:
            print(f"Translation error: {e}")
            return input_str # Return original string on error
    else:
        return "" # Return empty string for empty or None input

In [11]:
@F.udf(returnType=T.StringType())
def translate_udf(input_str): # udf, for dataframe columns.
    return translate(input_str)

In [12]:
# 1. Collect unique values
unique_fields = [x[0] for x in df_formacode.select("field").distinct().collect()]

In [13]:
# 2. Translate unique values
field_translations = {field: translate(field) for field in unique_fields if field is not None}

In [14]:
# 3. Broadcast dictionary
broadcast_fields = sc.broadcast(field_translations)

In [15]:
# 4. Create UDF to lookup values
@F.udf(returnType=T.StringType())
def lookup_translation(field):
    return broadcast_fields.value.get(field, "")

In [16]:
df_formacode = df_formacode.withColumn('description_en', translate_udf(F.col('description')))\
    .withColumn("field_en", lookup_translation(F.col("field")))

In [17]:
df_formacode.show()

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|formacode|         description|               field|        generic_term|      description_en|            field_en|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    46054|industrie graphiq...|460 INDUSTRIE GRA...|00104 TECHNOLOGIE...|Printing graphics...|460 Imprimerie gr...|
|    46072|       art graphique|460 INDUSTRIE GRA...|46054 INDUSTRIE G...|         graphic art|460 Imprimerie gr...|
|    46081|  création graphique|460 INDUSTRIE GRA...| 46072 ART GRAPHIQUE|    graphic creation|460 Imprimerie gr...|
|    46090| graphisme publicité|460 INDUSTRIE GRA...|46081 CREATION GR...|Advertising graphics|460 Imprimerie gr...|
|    46003|         infographie|460 INDUSTRIE GRA...|46081 CREATION GR...|         infographic|460 Imprimerie gr...|
|    46093|    chaîne graphique|460 INDUSTRIE GRA...|46054 INDUS

In [ ]:
df_formacode.show()

In [ ]:
df_formacode = df_formacode.repartition(16, "field")

In [18]:
print(f"Number of partitions: {df_formacode.rdd.getNumPartitions()}")

Number of partitions: 16


In [ ]:
print(f"Null field count: {df_formacode.filter(F.col('field').isNull()).count()}")
print(f"Empty field count: {df_formacode.filter(F.col('field') == '').count()}")

In [19]:
df_formacode.coalesce(1).write.parquet('gs://jugnu-france-course-enrollments/formacode_converted', mode='overwrite')

In [ ]:
import findspark
findspark.add_packages("com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.34.0")

In [ ]:
spark.conf.set('temporaryGcsBucket', 'jugnu-france-course-enrollments')

In [ ]:
# Update to your GCS bucket
gcs_bucket = 'dataproc-bucket-name'

# Update to your BigQuery dataset name you created
bq_dataset = 'dataset_name'

# Enter BigQuery table name you want to create or overwite. 
# If the table does not exist it will be created when you run the write function
bq_table = 'wiki_total_pageviews'

df_wiki_en_totals.write \
  .format("bigquery") \
  .option("table","{}.{}".format(bq_dataset, bq_table)) \
  .option("temporaryGcsBucket", gcs_bucket) \
  .mode('overwrite') \
  .save()

In [ ]:
spark_write_bigquery(data = df_formacode,
  projectId = "france-courses-enrollments",
  datasetID = "seed",
  tableID = "formacode",
  mode = "overwrite")

In [ ]:
df_formacode.write.format("bigquery") \
      .option("credentialsFile", "./gcs.json") \
      .option("table", "seed.formacode") \
      .save()

      #.option("temporaryGcsBucket", "jugnu-france-course-enrollments") \
      #.option("project", "france-courses-enrollments") \
      #.option("dataset", "seed") \

            #.mode("overwrite") \

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of table to append to.
table_id = "france-courses-enrollments.seed.test"

rows_to_insert = [
    {"full_name": "Phred Phlyntstone", "age": 32},
    {"full_name": "Wylma Phlyntstone", "age": 29},
]

errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
if errors == []:
    print("New rows have been added.")
else:
    print("Encountered errors while inserting rows: {}".format(errors))

In [ ]:
df_formacode.count()

In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame

def write_dataframe_to_bigquery(df: DataFrame, project_id: str, dataset_id: str, table_id: str, temp_gcs_bucket: str, gcs_credentials_path: str):
    """
    Writes a Spark DataFrame to BigQuery using a local GCS credentials JSON file.

    Args:
        df: The Spark DataFrame to write.
        project_id: The Google Cloud Project ID.
        dataset_id: The BigQuery dataset ID.
        table_id: The BigQuery table ID.
        temp_gcs_bucket: A Google Cloud Storage bucket for temporary storage.
        gcs_credentials_path: Path to the GCS credentials JSON file.
    """

    try:
        spark.conf.set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "./gcs.json")
        spark.conf.set("spark.hadoop.google.cloud.project", project_id) #add this line

        df.write \
            .format("bigquery") \
            .option("temporaryGcsBucket", temp_gcs_bucket) \
            .option("project", project_id) \
            .option("dataset", dataset_id) \
            .option("table", table_id) \
            .mode("overwrite") \
            .save()

        print(f"DataFrame written to BigQuery: {project_id}.{dataset_id}.{table_id}")

    except Exception as e:
        print(f"Error writing DataFrame to BigQuery: {e}")
        raise

# Example usage (replace with your actual values):
project_id = "france-courses-enrollments"
dataset_id = "seed"
table_id = "formacode"
temp_gcs_bucket = "jugnu-france-course-enrollments"
gcs_credentials_path = "./gcs.json"  # Path to your gcs.json file

# Assuming you already have a SparkSession created:
spark = SparkSession.builder.appName("WriteToBigQuery").getOrCreate()

# Assuming you have your DataFrame df_formacode:
# df_formacode = ... (your DataFrame creation logic)

# Sample dataframe creation for testing.
data = [("Alice", 34), ("Bob", 45), ("Charlie", 28)]
df_formacode = spark.createDataFrame(data, ["name", "age"])

#Ensure the gcs.json file exists
if not os.path.exists(gcs_credentials_path):
    raise FileNotFoundError(f"GCS credentials file not found: {gcs_credentials_path}")

write_dataframe_to_bigquery(df_formacode, project_id, dataset_id, table_id, temp_gcs_bucket, gcs_credentials_path)

#spark.stop()

In [ ]:

spark.stop()


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("VerifyConfig").getOrCreate()

spark.conf.set("spark.hadoop.google.cloud.project", "france-courses-enrollments")
print("Project ID from Spark Conf:", spark.conf.get("spark.hadoop.google.cloud.project"))

spark.conf.set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/./gcs.json") #use absolute path
print("Credentials Path from Spark Conf:", spark.conf.get("spark.hadoop.google.cloud.auth.service.account.json.keyfile"))

spark.stop()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BigQueryWrite")\
    .config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies:0.34.0")\
    .getOrCreate()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BigQueryReadTest").getOrCreate()

df = spark.read.format("bigquery") \
    .option("table", "france-courses-enrollments.staging.courses") \
    .load()
df.show()
spark.stop()

In [ ]:
import os
from pyspark.sql import SparkSession

# Set environment variables (do this at the VERY beginning of your notebook)
os.environ["GOOGLE_CLOUD_PROJECT"] = "france-courses-enrollments"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/jugnuarora/neuefische_self/france_courses_enrollments/gcs.json"  # Replace with your absolute path

# Verify environment variables (optional)
print("GOOGLE_CLOUD_PROJECT:", os.environ.get("GOOGLE_CLOUD_PROJECT"))
print("GOOGLE_APPLICATION_CREDENTIALS:", os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))

# Create SparkSession and run your BigQuery code
spark = SparkSession.builder.appName("BigQueryTest").getOrCreate()

# Example: Read from BigQuery (replace with your actual code)
try:
    df = spark.read.format("bigquery") \
        .option("table", "france-courses-enrollments.staging.courses") \
        .load()
    df.show()
except Exception as e:
    print(f"Error: {e}")

spark.stop()

S.date_extract, S.provider, S.department, S.region, S.type_referentiel, S.code_inventaire, S.code_rncp, S.certification_title, S.training_exit_level, S.code_formacode_1, S.code_formacode_2, S.code_formacode_3, S.code_formacode_4, S.code_formacode_5, S.main_formacode_desc, S.code_rome_1, S.code_rome_2, S.code_rome_3, S.code_rome_4, S.code_rome_5, S.nsf_code_1_desc, S.nsf_code_2_desc, S.nsf_code_3_desc, S.code_nsf_1, S.code_nsf_2 S.code_nsf_3, S.code_certifinfo, S.provider_ID, S.training_ID S.title, S.strengths, S.objectif_formation, S.contenu_formation, S.resultats_attendus_formation, S.nb_action, S.nb_active_session, S.nb_distant_session, S.duration_min, S.duration_max, S.duration_mean, S.cost_min, S.cost_max, S.cost_mean, S.code_departement, S.code_region, S.nbaction_nbheures, S.coderegion_export